# Data Prep for Jigsaw Multilabel

In [1]:
import sys
import os
from pathlib import Path
import getopt
import logging
import numpy as np
import pandas as pd
import msgpack
import re
import csv
from gensim.models import Word2Vec
from io import BytesIO
from tensorflow.python.lib.io import file_io

In [3]:
train_df = pd.read_csv('train.csv')

In [12]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
# =================================================== +
#                                                    /
#    P R E P A R E  W O R D  E M B E D D I N G S    /
#                                                  /
# ----------------------------------------------- +

# Build vocabulary and word embeddings from source.

csv_path = 'train.csv'

# Store recordss
all_labels = []              
tokens = []
ids = []
maxsentlen = 0
maxdoclen = 0
num_dropped = 0

# Process csv one line at a time
with open(csv_path, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    lineno = 0
    idx = 0
    for line in csv_reader:
        # Skip header.
        lineno += 1
        #sys.stdout.write("Processing line %i     \r" % lineno)
        #ssys.stdout.flush()

        # Begin at index 1.
        idx += 1

        text = line['comment_text']

        # Process raw text.

        # Remove unwanted tokens.
        text = re.sub('\n', ' ', text)
        text = re.sub('\ts', ' ', text)

        # Force lowercase.
        text = text.lower()

        # Remove single and double backticks.
        text = re.sub("`", '', text)

        # Remove single quotes.
        text = re.sub("'", '', text)

        # Replace multiple periods in sequence with one period.
        text = re.sub("\.{2,}", '.', text)

        # Replace everything except words, '.', '|', '?', and '!' with space.
        text = re.sub('[^\w_|\.|\?|!]+', ' ', text)

        # Replace periods with ' . '.
        text = re.sub('\.', ' . ', text)

        # Replace '?' with ' ? '.
        text = re.sub('\?', ' ? ', text)

        # Replace '!' with ' ! '.
        text = re.sub('!', ' ! ', text)

        # Tokenize by splitting on whitespace.
        # No leading or trailing whitespace is kept.
        # Consecutive spaces are treated as a single space.
        text = text.split()

        # Drop empty comments.
        if len(text) == 0:
            num_dropped += 1
            continue

        # Split into sentences.
        sentences = []
        sentence = []
        for t in text:
            # Use '.', '!', '?' as markers of end of sentence.
            if t not in ['.', '!', '?']:
                # Not at end of a sentence.
                sentence.append(t)
            else:
                # At end of a sentence.
                sentence.append(t)

                # Add sentence to sentences.
                sentences.append(sentence)

                # Track longest sentence.
                if len(sentence) > maxsentlen:
                    maxsentlen = len(sentence)

                # Reset sentence list.
                sentence = []

        # If sentence has word, add to list of sentences.
        if len(sentence) > 0:
            sentences.append(sentence)

        # Add split sentences to tokens.
        tokens.append(sentences)

        # Track longest document.
        if len(sentences) > maxdoclen:
            maxdoclen = len(sentences)

        # Build list of labels for record.
        doc_labels = []
        doc_labels.append(line['toxic'])
        doc_labels.append(line['severe_toxic'])
        doc_labels.append(line['obscene'])
        doc_labels.append(line['threat'])
        doc_labels.append(line['insult'])
        doc_labels.append(line['identity_hate'])

        # Add list of labels to list of all labels.
        all_labels.append(doc_labels)

        # Add id.
        ids.append(line['id'])